<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Read-in-USPTO-from-ORD" data-toc-modified-id="Read-in-USPTO-from-ORD-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Read in USPTO from ORD</a></span><ul class="toc-item"><li><span><a href="#Preface" data-toc-modified-id="Preface-1.1"><span class="toc-item-num">1.1&nbsp;&nbsp;</span>Preface</a></span></li><li><span><a href="#Extract-USPTO-data-from-ORD" data-toc-modified-id="Extract-USPTO-data-from-ORD-1.2"><span class="toc-item-num">1.2&nbsp;&nbsp;</span>Extract USPTO data from ORD</a></span></li><li><span><a href="#Tests:-Figure-out-how-to-access-the-info-I-need-in-the-dataset-file" data-toc-modified-id="Tests:-Figure-out-how-to-access-the-info-I-need-in-the-dataset-file-1.3"><span class="toc-item-num">1.3&nbsp;&nbsp;</span>Tests: Figure out how to access the info I need in the dataset file</a></span></li></ul></li><li><span><a href="#Preprocessing-of-USPTO---Molecular-AI" data-toc-modified-id="Preprocessing-of-USPTO---Molecular-AI-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Preprocessing of USPTO - Molecular AI</a></span><ul class="toc-item"><li><span><a href="#Read-in-data-cleaned-by-rxn-utils" data-toc-modified-id="Read-in-data-cleaned-by-rxn-utils-2.1"><span class="toc-item-num">2.1&nbsp;&nbsp;</span>Read in data cleaned by rxn utils</a></span></li></ul></li></ul></div>

# Read in USPTO from ORD

## Preface

In [ ]:
# I tried to read USPTO data from the ord schema, e.g. the data contained through this link:
# url = "https://github.com/Open-Reaction-Database/ord-data/blob/main/data/02/ord_dataset-026684a62f91469db49c7767d16c39fb.pb.gz?raw=true"
# However, ORD reads literally EVERYTHING from USPTO, so this resulted in around 90k x 120k df, which Joe's computer
# and my laptop do not have the momory to deal with.

# There may be 90k columns, but a lot of the columns may have superfluous info, e.g. a type column = SMILES, 
# email columns etc. 
# So one possible solution would be to pre-filter the columns (delete all the unnecessary ones), 
# and then load it afterwards

# I could use the code below to do this 
# However, it's unnecessary, as Joe is parsing the original USPTO xml files!


In [ ]:
# # import ord_schema
# # from ord_schema import message_helpers, validations
# # from ord_schema.proto import dataset_pb2

# # import wget

# # # url = "https://github.com/Open-Reaction-Database/ord-data/blob/main/data/02/ord_dataset-026684a62f91469db49c7767d16c39fb.pb.gz?raw=true"
# # url = "https://github.com/open-reaction-database/ord-data/blob/main/data/68/ord_dataset-68cb8b4b2b384e3d85b5b1efae58b203.pb.gz?raw=true"
# # pb = wget.download(url)

# # # Load Dataset message
# # data = message_helpers.load_message(pb, dataset_pb2.Dataset)

# rows = []
# for d in data.reactions:
#     # print(d)
#     row = message_helpers.message_to_row(d)
#     rows.append(row)
#     for k,v in row.items():
#         print(k)
#     break
# df = pd.DataFrame(rows)

In [ ]:
# Or following the example here
# https://github.com/open-reaction-database/ord-schema/blob/main/examples/applications/Perera_Science_Granda_Nature_Suzuki/Granda_Perera_ml_example.ipynb

# # Download dataset from ord-data
# url = "https://github.com/open-reaction-database/ord-data/blob/main/data/68/ord_dataset-68cb8b4b2b384e3d85b5b1efae58b203.pb.gz?raw=true"
# pb = wget.download(url)

# # Load Dataset message
# data = message_helpers.load_message(pb, dataset_pb2.Dataset)

# # Ensure dataset validates
# valid_output = validations.validate_message(data)

# # Convert dataset to pandas dataframe
# df = message_helpers.messages_to_dataframe(data.reactions, drop_constant_columns=True)

# # View dataframe
# df


# # View all columns with variation in the dataset
# list(df.columns)


## Extract USPTO data from ORD

1. All of the grants USPTO data is contained here: https://github.com/open-reaction-database/ord-data
2. It is batched by year, it's best to just maintain this batching, it will make it easier to handle (each file won't get excessively large)
3. Read in the data contained in the .pb.gz file, each entry in the "list" is a reaction. Write a for loop over the "list", and extract the following from each reaction:
    - Reactants
    - Products
    - Solvents
    - Reagents
    - Catalyst
    - Temperature
    - Yield
    - Anything else?
4. Build a list for each of these, combine to a df, and then save as a paraquet file
5. repeat this for each of the 41 years (41 datasets) we have data for in USPTO. It'll probably be easiest to convert the code in this notebook into a script, and then run it automatically on each.

In [ ]:
# Find the schema here
# https://github.com/open-reaction-database/ord-schema/blob/main/ord_schema/proto/reaction.proto

In [1]:
# Import modules
import ord_schema
from ord_schema import message_helpers, validations
from ord_schema.proto import dataset_pb2

import math
import pandas as pd
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
import os
import wget

from rdkit import Chem
from rdkit.Chem import AllChem
from sklearn import model_selection, metrics
from glob import glob

from tqdm import tqdm

2022-12-03 21:52:17.129875: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
# Download dataset from ord-data
#url = "https://github.com/open-reaction-database/ord-data/blob/main/data/68/ord_dataset-68cb8b4b2b384e3d85b5b1efae58b203.pb.gz?raw=true"
#https://github.com/open-reaction-database/ord-data
url = "https://github.com/Open-Reaction-Database/ord-data/blob/main/data/02/ord_dataset-026684a62f91469db49c7767d16c39fb.pb.gz?raw=true"
pb = wget.download(url)


  0% [                                                    ]        0 / 56336827
  0% [                                                    ]     8192 / 56336827
  0% [                                                    ]    16384 / 56336827
  0% [                                                    ]    24576 / 56336827
  0% [                                                    ]    32768 / 56336827
  0% [                                                    ]    40960 / 56336827
  0% [                                                    ]    49152 / 56336827
  0% [                                                    ]    57344 / 56336827
  0% [                                                    ]    65536 / 56336827
  0% [                                                    ]    73728 / 56336827
  0% [                                                    ]    81920 / 56336827
  0% [                                                    ]    90112 / 56336827
  0% [                                 


  6% [...                                                 ]  3596288 / 56336827
  6% [...                                                 ]  3604480 / 56336827
  6% [...                                                 ]  3612672 / 56336827
  6% [...                                                 ]  3620864 / 56336827
  6% [...                                                 ]  3629056 / 56336827
  6% [...                                                 ]  3637248 / 56336827
  6% [...                                                 ]  3645440 / 56336827
  6% [...                                                 ]  3653632 / 56336827
  6% [...                                                 ]  3661824 / 56336827
  6% [...                                                 ]  3670016 / 56336827
  6% [...                                                 ]  3678208 / 56336827
  6% [...                                                 ]  3686400 / 56336827
  6% [...                              


 13% [......                                              ]  7430144 / 56336827
 13% [......                                              ]  7438336 / 56336827
 13% [......                                              ]  7446528 / 56336827
 13% [......                                              ]  7454720 / 56336827
 13% [......                                              ]  7462912 / 56336827
 13% [......                                              ]  7471104 / 56336827
 13% [......                                              ]  7479296 / 56336827
 13% [......                                              ]  7487488 / 56336827
 13% [......                                              ]  7495680 / 56336827
 13% [......                                              ]  7503872 / 56336827
 13% [......                                              ]  7512064 / 56336827
 13% [......                                              ]  7520256 / 56336827
 13% [......                           


 19% [..........                                          ] 11141120 / 56336827
 19% [..........                                          ] 11149312 / 56336827
 19% [..........                                          ] 11157504 / 56336827
 19% [..........                                          ] 11165696 / 56336827
 19% [..........                                          ] 11173888 / 56336827
 19% [..........                                          ] 11182080 / 56336827
 19% [..........                                          ] 11190272 / 56336827
 19% [..........                                          ] 11198464 / 56336827
 19% [..........                                          ] 11206656 / 56336827
 19% [..........                                          ] 11214848 / 56336827
 19% [..........                                          ] 11223040 / 56336827
 19% [..........                                          ] 11231232 / 56336827
 19% [..........                       


 27% [..............                                      ] 15269888 / 56336827
 27% [..............                                      ] 15278080 / 56336827
 27% [..............                                      ] 15286272 / 56336827
 27% [..............                                      ] 15294464 / 56336827
 27% [..............                                      ] 15302656 / 56336827
 27% [..............                                      ] 15310848 / 56336827
 27% [..............                                      ] 15319040 / 56336827
 27% [..............                                      ] 15327232 / 56336827
 27% [..............                                      ] 15335424 / 56336827
 27% [..............                                      ] 15343616 / 56336827
 27% [..............                                      ] 15351808 / 56336827
 27% [..............                                      ] 15360000 / 56336827
 27% [..............                   

...


 64% [.................................                   ] 36536320 / 56336827
 64% [.................................                   ] 36544512 / 56336827
 64% [.................................                   ] 36552704 / 56336827
 64% [.................................                   ] 36560896 / 56336827
 64% [.................................                   ] 36569088 / 56336827
 64% [.................................                   ] 36577280 / 56336827
 64% [.................................                   ] 36585472 / 56336827
 64% [.................................                   ] 36593664 / 56336827
 64% [.................................                   ] 36601856 / 56336827
 64% [.................................                   ] 36610048 / 56336827
 64% [.................................                   ] 36618240 / 56336827
 65% [.................................                   ] 36626432 / 56336827
 65% [.................................


 72% [.....................................               ] 40828928 / 56336827
 72% [.....................................               ] 40837120 / 56336827
 72% [.....................................               ] 40845312 / 56336827
 72% [.....................................               ] 40853504 / 56336827
 72% [.....................................               ] 40861696 / 56336827
 72% [.....................................               ] 40869888 / 56336827
 72% [.....................................               ] 40878080 / 56336827
 72% [.....................................               ] 40886272 / 56336827
 72% [.....................................               ] 40894464 / 56336827
 72% [.....................................               ] 40902656 / 56336827
 72% [.....................................               ] 40910848 / 56336827
 72% [.....................................               ] 40919040 / 56336827
 72% [.................................


 79% [.........................................           ] 44859392 / 56336827
 79% [.........................................           ] 44867584 / 56336827
 79% [.........................................           ] 44875776 / 56336827
 79% [.........................................           ] 44883968 / 56336827
 79% [.........................................           ] 44892160 / 56336827
 79% [.........................................           ] 44900352 / 56336827
 79% [.........................................           ] 44908544 / 56336827
 79% [.........................................           ] 44916736 / 56336827
 79% [.........................................           ] 44924928 / 56336827
 79% [.........................................           ] 44933120 / 56336827
 79% [.........................................           ] 44941312 / 56336827
 79% [.........................................           ] 44949504 / 56336827
 79% [.................................


 87% [.............................................       ] 49373184 / 56336827
 87% [.............................................       ] 49381376 / 56336827
 87% [.............................................       ] 49389568 / 56336827
 87% [.............................................       ] 49397760 / 56336827
 87% [.............................................       ] 49405952 / 56336827
 87% [.............................................       ] 49414144 / 56336827
 87% [.............................................       ] 49422336 / 56336827
 87% [.............................................       ] 49430528 / 56336827
 87% [.............................................       ] 49438720 / 56336827
 87% [.............................................       ] 49446912 / 56336827
 87% [.............................................       ] 49455104 / 56336827
 87% [.............................................       ] 49463296 / 56336827
 87% [.................................


 93% [................................................    ] 52731904 / 56336827
 93% [................................................    ] 52740096 / 56336827
 93% [................................................    ] 52748288 / 56336827
 93% [................................................    ] 52756480 / 56336827
 93% [................................................    ] 52764672 / 56336827
 93% [................................................    ] 52772864 / 56336827
 93% [................................................    ] 52781056 / 56336827
 93% [................................................    ] 52789248 / 56336827
 93% [................................................    ] 52797440 / 56336827
 93% [................................................    ] 52805632 / 56336827
 93% [................................................    ] 52813824 / 56336827
 93% [................................................    ] 52822016 / 56336827
 93% [.................................

In [2]:
# Load Dataset message
pb = 'data/USPTO/ord_dataset-026684a62f91469db49c7767d16c39fb.pb.gz'
data = message_helpers.load_message(pb, dataset_pb2.Dataset)

In [19]:
valid_output = validations.validate_message(data)

[20:57:50] reactant 3 has no mapped atoms.
[20:57:50] reactant 4 has no mapped atoms.
[20:57:51] reactant 2 has no mapped atoms.
[20:57:51] reactant 3 has no mapped atoms.
[20:57:51] reactant 5 has no mapped atoms.
[20:57:51] reactant 6 has no mapped atoms.
[20:57:51] reactant 0 has no mapped atoms.
[20:57:51] reactant 2 has no mapped atoms.
[20:57:51] reactant 3 has no mapped atoms.
[20:57:51] reactant 5 has no mapped atoms.
[20:57:51] reactant 0 has no mapped atoms.
[20:57:51] reactant 2 has no mapped atoms.
[20:57:51] reactant 3 has no mapped atoms.
[20:57:51] reactant 5 has no mapped atoms.
[20:57:51] reactant 1 has no mapped atoms.
[20:57:51] reactant 2 has no mapped atoms.
[20:57:51] reactant 3 has no mapped atoms.
[20:57:51] reactant 5 has no mapped atoms.
[20:57:51] reactant 0 has no mapped atoms.
[20:57:51] reactant 2 has no mapped atoms.
[20:57:51] reactant 4 has no mapped atoms.
[20:57:51] reactant 5 has no mapped atoms.
[20:57:51] product atom-mapping number 4 found multipl

[20:57:51] reactant 2 has no mapped atoms.
[20:57:51] reactant 3 has no mapped atoms.
[20:57:51] reactant 4 has no mapped atoms.
[20:57:51] reactant 5 has no mapped atoms.
[20:57:51] reactant 6 has no mapped atoms.
[20:57:51] reactant 2 has no mapped atoms.
[20:57:51] reactant 3 has no mapped atoms.
[20:57:51] reactant 4 has no mapped atoms.
[20:57:51] reactant 5 has no mapped atoms.
[20:57:51] reactant 2 has no mapped atoms.
[20:57:51] reactant 1 has no mapped atoms.
[20:57:51] reactant 2 has no mapped atoms.
[20:57:51] reactant 3 has no mapped atoms.
[20:57:51] reactant 4 has no mapped atoms.
[20:57:51] reactant 5 has no mapped atoms.
[20:57:51] reactant 1 has no mapped atoms.
[20:57:51] reactant 1 has no mapped atoms.
[20:57:51] product atom-mapping number 24 found multiple times.
[20:57:51] product atom-mapping number 25 found multiple times.
[20:57:51] product atom-mapping number 28 found multiple times.
[20:57:51] product atom-mapping number 27 found multiple times.
[20:57:51] pr

[20:57:53] reactant 3 has no mapped atoms.
[20:57:53] reactant 4 has no mapped atoms.
[20:57:53] reactant 5 has no mapped atoms.
[20:57:53] reactant 1 has no mapped atoms.
[20:57:53] reactant 2 has no mapped atoms.
[20:57:53] reactant 2 has no mapped atoms.
[20:57:53] reactant 1 has no mapped atoms.
[20:57:53] reactant 3 has no mapped atoms.
[20:57:53] reactant 4 has no mapped atoms.
[20:57:53] reactant 5 has no mapped atoms.
[20:57:53] reactant 7 has no mapped atoms.
[20:57:53] reactant 2 has no mapped atoms.
[20:57:53] reactant 2 has no mapped atoms.
[20:57:53] reactant 1 has no mapped atoms.
[20:57:53] reactant 2 has no mapped atoms.
[20:57:53] reactant 2 has no mapped atoms.
[20:57:53] reactant 3 has no mapped atoms.
[20:57:53] reactant 4 has no mapped atoms.
[20:57:53] reactant 5 has no mapped atoms.
[20:57:53] reactant 1 has no mapped atoms.
[20:57:53] reactant 1 has no mapped atoms.
[20:57:53] reactant 3 has no mapped atoms.
[20:57:53] reactant 4 has no mapped atoms.
[20:57:53] 

[20:57:53] reactant 2 has no mapped atoms.
[20:57:53] reactant 1 has no mapped atoms.
[20:57:53] reactant 2 has no mapped atoms.
[20:57:53] reactant 3 has no mapped atoms.
[20:57:53] reactant 4 has no mapped atoms.
[20:57:53] reactant 2 has no mapped atoms.
[20:57:53] reactant 1 has no mapped atoms.
[20:57:53] reactant 2 has no mapped atoms.
[20:57:53] reactant 3 has no mapped atoms.
[20:57:53] reactant 4 has no mapped atoms.
[20:57:53] reactant 2 has no mapped atoms.
[20:57:53] reactant 3 has no mapped atoms.
[20:57:53] reactant 1 has no mapped atoms.
[20:57:53] reactant 2 has no mapped atoms.
[20:57:53] reactant 3 has no mapped atoms.
[20:57:53] reactant 4 has no mapped atoms.
[20:57:53] reactant 2 has no mapped atoms.
[20:57:53] reactant 1 has no mapped atoms.
[20:57:53] reactant 2 has no mapped atoms.
[20:57:54] reactant 1 has no mapped atoms.
[20:57:54] reactant 2 has no mapped atoms.
[20:57:54] reactant 1 has no mapped atoms.
[20:57:54] reactant 3 has no mapped atoms.
[20:57:54] 

[20:57:54] reactant 2 has no mapped atoms.
[20:57:54] reactant 3 has no mapped atoms.
[20:57:54] reactant 4 has no mapped atoms.
[20:57:54] reactant 5 has no mapped atoms.
[20:57:54] reactant 6 has no mapped atoms.
[20:57:54] reactant 1 has no mapped atoms.
[20:57:54] reactant 2 has no mapped atoms.
[20:57:54] reactant 3 has no mapped atoms.
[20:57:54] reactant 4 has no mapped atoms.
[20:57:54] reactant 2 has no mapped atoms.
[20:57:54] reactant 2 has no mapped atoms.
[20:57:55] reactant 2 has no mapped atoms.
[20:57:55] reactant 1 has no mapped atoms.
[20:57:55] reactant 2 has no mapped atoms.
[20:57:55] reactant 3 has no mapped atoms.
[20:57:55] reactant 1 has no mapped atoms.
[20:57:55] reactant 2 has no mapped atoms.
[20:57:55] reactant 4 has no mapped atoms.
[20:57:55] reactant 5 has no mapped atoms.
[20:57:55] reactant 6 has no mapped atoms.
[20:57:55] WARNING: not removing hydrogen atom without neighbors
[20:57:55] WARNING: not removing hydrogen atom without neighbors
[20:57:55]

...

[20:58:03] reactant 0 has no mapped atoms.
[20:58:03] reactant 2 has no mapped atoms.
[20:58:03] product atom-mapping number 8 found multiple times.
[20:58:03] product atom-mapping number 12 found multiple times.
[20:58:03] product atom-mapping number 13 found multiple times.
[20:58:03] product atom-mapping number 15 found multiple times.
[20:58:03] product atom-mapping number 19 found multiple times.
[20:58:03] product atom-mapping number 20 found multiple times.
[20:58:03] product atom-mapping number 21 found multiple times.
[20:58:03] product atom-mapping number 22 found multiple times.
[20:58:03] product atom-mapping number 23 found multiple times.
[20:58:03] product atom-mapping number 24 found multiple times.
[20:58:03] product atom-mapping number 27 found multiple times.
[20:58:03] product atom-mapping number 28 found multiple times.
[20:58:03] product atom-mapping number 29 found multiple times.
[20:58:03] product atom-mapping number 30 found multiple times.
[20:58:03] product 

[20:58:04] reactant 1 has no mapped atoms.
[20:58:04] reactant 2 has no mapped atoms.
[20:58:04] product atom-mapping number 1 found multiple times.
[20:58:04] product atom-mapping number 16 found multiple times.
[20:58:04] product atom-mapping number 17 found multiple times.
[20:58:04] product atom-mapping number 18 found multiple times.
[20:58:04] product atom-mapping number 19 found multiple times.
[20:58:04] product atom-mapping number 20 found multiple times.
[20:58:04] product atom-mapping number 21 found multiple times.
[20:58:04] product atom-mapping number 2 found multiple times.
[20:58:04] product atom-mapping number 3 found multiple times.
[20:58:04] product atom-mapping number 4 found multiple times.
[20:58:04] product atom-mapping number 7 found multiple times.
[20:58:04] product atom-mapping number 8 found multiple times.
[20:58:04] product atom-mapping number 12 found multiple times.
[20:58:04] product atom-mapping number 11 found multiple times.
[20:58:04] product atom-

[20:58:04] reactant 1 has no mapped atoms.
[20:58:04] reactant 0 has no mapped atoms.
[20:58:04] reactant 2 has no mapped atoms.
[20:58:04] reactant 3 has no mapped atoms.
[20:58:04] reactant 4 has no mapped atoms.
[20:58:04] reactant 2 has no mapped atoms.
[20:58:04] reactant 3 has no mapped atoms.
[20:58:04] reactant 4 has no mapped atoms.
[20:58:04] reactant 1 has no mapped atoms.
[20:58:04] reactant 2 has no mapped atoms.
[20:58:04] reactant 1 has no mapped atoms.
[20:58:04] reactant 2 has no mapped atoms.
[20:58:04] reactant 1 has no mapped atoms.
[20:58:04] reactant 2 has no mapped atoms.
[20:58:04] reactant 1 has no mapped atoms.
[20:58:04] reactant 2 has no mapped atoms.
[20:58:04] reactant 1 has no mapped atoms.
[20:58:04] reactant 2 has no mapped atoms.
[20:58:04] reactant 2 has no mapped atoms.
[20:58:04] reactant 3 has no mapped atoms.
[20:58:04] reactant 2 has no mapped atoms.
[20:58:04] reactant 3 has no mapped atoms.
[20:58:04] reactant 2 has no mapped atoms.
[20:58:04] 

[20:58:05] reactant 1 has no mapped atoms.
[20:58:05] reactant 2 has no mapped atoms.
[20:58:05] reactant 4 has no mapped atoms.
[20:58:05] reactant 5 has no mapped atoms.
[20:58:05] WARNING: not removing hydrogen atom without neighbors
[20:58:05] WARNING: not removing hydrogen atom without neighbors
[20:58:05] reactant 2 has no mapped atoms.
[20:58:05] reactant 3 has no mapped atoms.
[20:58:05] reactant 2 has no mapped atoms.
[20:58:05] reactant 2 has no mapped atoms.
[20:58:05] reactant 2 has no mapped atoms.
[20:58:05] reactant 3 has no mapped atoms.
[20:58:05] reactant 0 has no mapped atoms.
[20:58:05] reactant 2 has no mapped atoms.
[20:58:05] reactant 4 has no mapped atoms.
[20:58:05] reactant 5 has no mapped atoms.
[20:58:06] reactant 0 has no mapped atoms.
[20:58:06] reactant 1 has no mapped atoms.
[20:58:06] reactant 2 has no mapped atoms.
[20:58:06] reactant 3 has no mapped atoms.
[20:58:06] reactant 4 has no mapped atoms.
[20:58:06] reactant 2 has no mapped atoms.
[20:58:06]

ValidationError: Dataset.reactions[1478].inputs["m1_m2"].components[1].identifiers[2]: RDKit 2022.03.5 could not validate InChI identifier InChI=1S/CHCl/c1-2/h1H

In [ ]:
# inputs
# REACTANT = 1;
# REAGENT = 2;
# SOLVENT = 3;
# CATALYST = 4;
# WORKUP = 5;
# INTERNAL_STANDARD = 6;
# AUTHENTIC_STANDARD = 7;
# PRODUCT = 8;

# temperature:
# UNSPECIFIED = 0;
# CUSTOM = 1;
# AMBIENT = 2;
# OIL_BATH = 3;
# WATER_BATH = 4;
# SAND_BATH = 5;
# ICE_BATH = 6;
# DRY_ALUMINUM_PLATE = 7;
# MICROWAVE = 8;
# DRY_ICE_BATH = 9;
# AIR_FAN = 10;
# LIQUID_NITROGEN = 11;

# structure
# inputs -> m1, m2, m3 ...
# conditions -> temperature, ...
# notes
# workups
# outcomes -> reactants, yield?

In [68]:
#Known problems
# Some reactants (probably also other categories) only have the 'name' identifier, and thus, it is not possble to extract the smiles using [1] syntax
#   I'd need to write a try statement, and if we can't extract the smiles, then extract the name. Then we need to resolve the name and return a SMILES

reactants_all = []
reagents_all = []
products_all = []
solvents_all = []
reagents_all = []
catalysts_all = []

temperature_all = []

rxn_times_all = []

for i in tqdm(range(len(data.reactions))):
    rxn = data.reactions[i]
    # handle rxn inputs: reactants, reagents etc
    reactants = []
    reagents = []
    solvents = []
    catalysts = []
    products = []
    
    temperatures = []

    rxn_times = []

    # inputs
    for key in rxn.inputs: #these are the keys in the 'dict' style data struct
        try:
            rxn_role = rxn.inputs[key].components[0].reaction_role #rxn role
            if rxn_role == 1: #reactant
                reactants += [rxn.inputs[key].components[0].identifiers[1].value] # this should be the smiles
            elif rxn_role ==2: #reagent
                reagents += [rxn.inputs[key].components[0].identifiers[1].value] # this should be the smiles
            elif rxn_role ==3: #solvent
                solvents += [rxn.inputs[key].components[0].identifiers[1].value] # this should be the smiles
            elif rxn_role ==4: #catalyst
                catalysts += [rxn.inputs[key].components[0].identifiers[1].value] # this should be the smiles
            elif rxn_role in [5,6,7]: #workup, internal standard, authentic standard. don't care about these
                continue
            elif rxn_role ==8: #product
                products += [rxn.inputs[key].components[0].identifiers[1].value] # this should be the smiles
        except IndexError:
            #print(i, key )
            continue

    # temperature
    try:
        temperatures +=[rxn.conditions.temperature.control.type]
    except IndexError:
        temperatures += [np.nan]

    #outcomes
    try:
        rxn_times = (rxn.outcomes[0].reaction_time.value, rxn.outcomes[0].reaction_time.units)
        products = [rxn.outcomes[0].products[0].identifiers[1].value]
    except IndexError:
        rxn_times = (np.nan, np.nan)
        products = [np.nan]



    reactants_all += [reactants]
    reagents_all += [reagents]
    solvents_all += [solvents]
    catalysts_all += [catalyst]
    
    temperature_all = [temperatures]

    rxn_times_all += [rxn_times]      
    products_all += [products]
              

  5%|▍         | 4266/93834 [00:00<00:04, 20005.80it/s]

0 m4
1 m4_5
1 m4_m5
5 m4
39 m2_m3_m4_m5_m7_m6_m8
98 m4
167 m2
178 m3
185 m4_m6
243 m3_m6
245 m6_m7
305 m6_m9
310 m1
321 m3
437 m3
466 m2
551 m1_m2
568 m4
638 m2
649 m3
656 m4_m6
680 m6_m10
692 m1
692 m3
699 m4
700 m5_m1_m2_m3
701 m5_m1_m2_m3
702 m5_m1_m2_m3
703 m5_m1_m2_m3
704 m5_m1_m2_m3
705 m5_m1_m2_m3
707 m2_m3
708 m3_m4
724 m2
729 m3_m4
737 m1_m2_m4
739 m2
740 m3_m4_m6
773 m9
773 m1
776 m1_m4_m2_m3
778 m3
805 m3
806 m3
893 m1_m2_m3_m4
894 m5
907 m6_m7_m8
913 m7_m8_m9
914 m7_m8_m9
918 m8_m9
919 m1_m2_m3_m5
920 m3
920 m4
921 m3
922 m3
931 m4
942 m3
948 m3
966 m1_m2_m4_m3
998 m3_m6
1002 m2_m3_m4_m5
1030 m1
1051 m4
1053 m2
1087 m4
1101 m5
1115 m7
1190 m1_m2_m5_m3
1192 m1_m2_m3
1193 m1_m2_m3
1194 m1_m2_m5
1195 m1_m2_m5
1275 m1_m2_m3_m6_m4_m7
1281 m7_m4
1287 m1_m2_m3_m4_m5
1298 m6
1298 m3
1303 m4
1304 m4
1315 m1_m2_m4
1316 m7
1320 m3
1321 m5
1326 m3
1327 m3
1328 m3
1329 m3
1338 m3
1339 m3
1340 m3
1341 m3
1343 m3
1344 m3
1345 m3
1346 m3
1347 m3
1348 m3
1349 m3
1350 m3
1353 m3
1354 m3
1355

  7%|▋         | 6337/93834 [00:00<00:05, 16988.32it/s]

 m3
4344 m3
4347 m3
4356 m3
4359 m3
4370 m6_m8
4390 m4
4390 m3
4401 m2
4413 m4
4419 m3_m6_m4
4421 m3
4427 m3
4448 m2
4453 m2_m3
4464 m5
4482 m3
4482 m6_m7_m8
4542 m1_m2_m3_m5
4671 m2
4680 m3
4769 m2
4772 m2
4846 m5_m6
4851 m1_m2_m3
4852 m1_m2_m3
4854 m1_m2_m3
4855 m1_m2_m3
4862 m1_m2
4956 m4
4998 m1_m2_m3
5000 m1_m2_m3
5002 m1
5003 m2
5004 m1
5004 m3
5005 m1
5006 m2
5009 m2
5011 m3
5012 m3
5012 m5
5014 m1_m2_m3
5016 m1_m2
5022 m1_m2
5022 m3
5023 m6
5026 m3_m4
5026 m1
5026 m5
5059 m4
5060 m3
5142 m3_m4_m9
5146 m7_m4
5146 m5
5147 m5_m4
5165 m5
5165 m3
5174 m7_m1_m2_m3_m8_m9
5254 m1_m2_m7
5254 m4_m5_m6
5257 m2_m3_m7
5257 m4_m5_m8
5259 m2_m3_m7_m4_m5
5260 m1_m10_m2_m3_m6_m7
5278 m6
5285 m2_m9_m3_m4_m7
5361 m1_m6_m2_m3_m8
5365 m2_m11_m3_m4_m7_m8
5371 m3
5373 m2_m11_m3_m4_m7_m8
5382 m2
5398 m4
5436 m3
5440 m4_m5
5442 m3_m4_m5
5470 m2
5472 m5
5486 m4_m5
5518 m2
5520 m5
5556 m3
5579 m4_m5
5582 m4
5584 m5
5620 m4
5715 m3_m4
5722 m3
5787 m4
5976 m2
5983 m2
6053 m3
6104 m3
6120 m1
6120 m2
6131 m2

 11%|█▏        | 10736/93834 [00:00<00:05, 15988.64it/s]

...

 m2
83699 m4
83717 m3_m4
83726 m5_m6
83737 m3_m5
83739 m6_m7
83742 m5_m6
83777 m3
83777 m1_m6_m2_m7
83880 m4
83882 m3
83896 m3
83898 m4
83902 m3_m4
83906 m3
83931 m2
83931 m1
83932 m1
83932 m2
83933 m3
83934 m3
83935 m3
83938 m5
83938 m6
83939 m5
83948 m6_m8
83972 m4_m8_m5
83972 m2_m7_m3
83973 m2
83977 m1_m2_m5
83979 m4
83979 m3
83985 m4_m8
83996 m2_m3_m5
84010 m4
84017 m4
84033 m3
84033 m7
84034 m3
84037 m7
84037 m3
84038 m2
84038 m5
84038 m4
84046 m2_m3_m5
84066 m4
84068 m5
84072 m4
84092 m5
84092 m1_m2_m4
84099 m4_m10
84101 m7
84101 m8
84109 m1_m2_m5_m3_m6
84127 m1_m2_m5_m3_m6
84133 m4
84139 m4
84155 m1_m2_m7
84174 m4
84200 m1_m2_m5_m3_m6
84214 m4
84227 m3_m6_m7_m4
84229 m1_m2_m3_m5
84283 m5
84291 m5
84304 m5
84310 m5
84317 m5
84329 m5
84342 m5
84345 m5
84358 m6_m4
84367 m4
84372 m4
84380 m4_m5
84403 m3
84405 m3
84504 m1
84506 m1
84536 m9_m10
84553 m1_m2_m3
84556 m6
84573 m1_m2_m3
84574 m1_m2_m3
84576 m1_m2_m3
84577 m1_m2
84593 m1_m5
84606 m1_m2_m3
84607 m1_m2_m3
84610 m1_m2_m3_m4_m

 95%|█████████▌| 89548/93834 [00:06<00:00, 16948.80it/s]

 m2_m3
87661 m2_m3
87663 m2_m3
87664 m2_m3
87671 m2_m3
87678 m6
87681 m1
87682 m1
87682 m6
87683 m1
87685 m2
87686 m2
87711 m5
87749 m3
87754 m3
87755 m3
87785 m1_m2_m3_m11_m4_m5_m10_m9
87785 m6_m7
87787 m4
87787 m6
87794 m6
87817 m1_m2_m4
87819 m1_m2_m3_m5
87843 m4
87845 m10_m4_m5
87847 m10_m4_m5
87851 m6
87916 m3_m7_m4_m5_m6_m9
87922 m4
87937 m4
87960 m2
87961 m2
87962 m2
87964 m2
87965 m2
87966 m2
87967 m1
87970 m1_m2_m3_m4_m5
87996 m7_m5
88058 m2_m3_m4_m5_m10
88073 m2_m3
88118 m5
88123 m5
88128 m4
88148 m5_m0
88155 m1_m4_m2_m3_m5
88157 m2_m6
88170 m3
88194 m3
88213 m5
88213 m6
88220 m3
88221 m3
88222 m3
88223 m3
88227 m2_m3
88228 m3
88236 m3
88288 m3_m10
88288 m4_m5_m6
88288 m7
88297 m3
88314 m3_m4_m7
88318 m4
88329 m6
88330 m4_m5
88333 m1
88357 m4
88369 m1
88385 m3_m4_m6
88412 m4
88426 m1_m2_m4_m3_m5
88435 m1_m2_m4_m3_m5
88456 m1_m2_m5_m3_m6
88461 m2_m3_m5
88462 m2_m3_m10
88462 m5_m6_m9
88464 m8
88465 m1_m2_m5
88468 m1_m2_m3_m4_m6_m7
88469 m3
88477 m3
88480 m3
88481 m2_m3_m5
88482

100%|██████████| 93834/93834 [00:06<00:00, 14978.73it/s]

m5
91222 m5
91226 m0_m4
91229 m3
91245 m1_m4_m2
91248 m3_m4
91249 m2
91251 m3
91257 m2_m3_m7_m4
91264 m2_m3_m4
91268 m2_m3_m4
91276 m3
91279 m3
91281 m3
91283 m3
91284 m3_m8_m4
91302 m1_m2_m6
91331 m3_m4_m5_m6
91353 m5
91353 m2
91366 m4
91431 m4
91432 m4
91434 m4
91435 m3
91436 m4
91467 m4
91468 m4
91470 m4
91471 m3
91473 m4
91499 m3
91506 m2_m3_m5_m4
91509 m5
91510 m6
91511 m1
91565 m1_m6
91567 m1_m6
91567 m3_m4
91569 m3_m4
91569 m1_m6
91571 m1_m2_m4_m3_m5
91572 m3_m4
91572 m1_m5
91576 m6
91580 m6
91581 m5
91584 m5
91589 m3
91596 m4
91602 m1_m2_m6
91602 m3
91606 m7_m6_m0
91613 m8_m7_m0
91615 m4
91616 m2
91617 m9_m6_m0
91623 m1_m2_m5
91624 m1_m2_m7
91634 m1_m7
91640 m4
91640 m1_m8
91653 m2_m5
91654 m6
91658 m0_m1_m3
91659 m1_m5_m2
91661 m13
91661 m6_m7_m11_m12_m8_m9
91674 m2
91674 m3
91723 m4
91724 m3
91738 m1_m2_m3
91755 m4
91760 m1_m2
91768 m6_m7_m9
91768 m4
91794 m2_m6_m3
91810 m4
91814 m4
91818 m4
91821 m4
91823 m4
91835 m1
91836 m1
91837 m1
91838 m1
91841 m3_m4
91841 m6
91843 m6
9

In [36]:
for i in products_all:
    if i:
        print(i)

In [3]:
data.name

'uspto-grants-2016'

In [4]:
res = [field.name for field in data.DESCRIPTOR.fields]
res
# can do data.name to get the year, e.g. data.name returns 'uspto-grants-2016'
# can do data.dataset_id to get the ord file name, e.g. 'ord_dataset-026684a62f91469db49c7767d16c39fb'

['name', 'description', 'reactions', 'reaction_ids', 'dataset_id']

In [7]:
1+1

2

## Tests: Figure out how to access the info I need in the dataset file

In [62]:
data.reactions[3].outcomes[0].products[0].identifiers[1].value

'NC=1C(C=C(C(C1)=N)OCCCC)=NC1=CC=C(C=C1)N(CCO)CCO'

In [70]:
#91434 m4
data.reactions[91434].inputs['m4']

components {
  identifiers {
    type: NAME
    value: "Hexanes EtOAc"
  }
  amount {
    moles {
      value: 0.0
      precision: 1.0
      units: MOLE
    }
  }
  reaction_role: REACTANT
}

In [ ]:
# structure
# inputs -> m1, m2, m3 ...
# conditions -> temperature, ...
# notes
# workups
# outcomes -> reactants, yield?


In [24]:
rxn.inputs['m1'].components[0].reaction_role

1

In [34]:
rxn.inputs['m1'].components[1].identifiers[0].value

'ethyl 4-bromo-1-(3,4-difluorophenyl)-1H-pyrazole-3-carboxylate'

In [6]:
rxn.conditions.temperature.control.type

2

# Preprocessing of USPTO - Molecular AI

In [1]:
# Running code from:
# https://molecularai.github.io/reaction_utils/uspto.html

# From within the folder containing all the USPTO data

# First run:
# conda activate <rxnutilities>
#  python -m rxnutils.data.uspto.preparation_pipeline run --nbatches 200  --max-workers 8 --max-num-splits 200

# Then I was supposed to run:
# conda activate rxnmapper
# python -m rxnutils.data.mapping_pipeline run --data-prefix uspto --nbatches 200  --max-workers 8 --max-num-splits 200
# But that didn't work, so I just ran the first part 
# Even after I replaced the delimiter (from 	 to , it still failed ). I'll just give up lol

## Read in data cleaned by rxn utils

In [1]:
import pandas as pd

In [2]:
cleaned_USPTO = pd.read_csv('/Users/dsw46/USPTO_data/uspto_data_cleaned.csv', sep = '	')
cleaned_USPTO.shape

(3740596, 7)

In [3]:
cleaned_USPTO['ReactionSmilesClean'][0]

'OCCBr.CCS(=O)(=O)Cl.CCOCC.CCN(CC)CC>>CCS(=O)(=O)OCCBr'

In [4]:
#full_USPTO = pd.read_csv('/Users/dsw46/USPTO_data/uspto_data.csv', sep = '	')
#full_USPTO.shape